In [11]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd

def make_adp_df(BASE_URL = "https://www.fantasypros.com/nfl/adp/half-point-ppr-overall.php"):
    res = requests.get(BASE_URL)
    if res.ok:
        soup = BS(res.content, 'html.parser')
        table = soup.find('table', {'id': 'data'})
        df = pd.read_html(str(table))[0]
#         print('Output after reading the html:\n\n', df.head(), '\n') # so you can see the output at this point 
        df = df[['Player Team (Bye)', 'POS', 'AVG']]
#         print('Output after filtering:\n\n', df.head(), '\n')
        df['PLAYER'] = df['Player Team (Bye)'].apply(lambda x: ' '.join(x.split()[:-2])) # removing the team and position
        df['POS'] = df['POS'].apply(lambda x: x[:2]) # removing the position rank
        
        df = df[['PLAYER', 'POS', 'AVG']].sort_values(by='AVG')
        
#         print('Final output: \n\n', df.head())
        
        return df
        
    else:
        print('oops, something didn\'t work right', res.status_code)
        
df = make_adp_df()

replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
}

for _, row in df[:100].iterrows():
    position = row['POS']
    player = row['PLAYER']
    replacement_players[position] = player
    
# each position has a different associated URL. We'll create a string format here and loop through the possible positions

def make_projection_df(BASE_URL = 'https://www.fantasypros.com/nfl/projections/{position}.php?week=draft'):
    
    # we are going to concatenate our individual position dfs into this larger final_df
    final_df = pd.DataFrame()
    
    #url has positions in lower case
    for position in ['rb', 'qb', 'te', 'wr']:
        
        res = requests.get(BASE_URL.format(position=position)) # format our url with the position
        if res.ok:
            soup = BS(res.content, 'html.parser')
            table = soup.find('table', {'id': 'data'})
            df = pd.read_html(str(table))[0]
            
            df.columns = df.columns.droplevel(level=0) # our data has a multi-level column index. The first column level is useless so let's drop it.
            df['PLAYER'] = df['Player'].apply(lambda x: ' '.join(x.split()[:-1])) # fixing player name to not include team
            # if you're not doing PPR, don't include this. If you're doing Half PPR,
            # multiply receptions * 1/2
            if 'REC' in df.columns:
                df['FPTS'] = df['FPTS'] + df['REC'] # add receptions if they're in there. 
            
            df['POS'] = position.upper() # add a position column
            
            df = df[['PLAYER', 'POS', 'FPTS']]
            final_df = pd.concat([final_df, df]) # iteratively add to our final_df
        else:
            print('oops something didn\'t work right', res.status_code)
            return
    
    final_df = final_df.sort_values(by='FPTS', ascending=False) # sort df in descending order on FPTS column
    
    return final_df
            
df = make_projection_df()

replacement_values = {
    'RB': 0,
    'WR': 0,
    'QB': 0,
    'TE': 0
}

for position, player in replacement_players.items():
    if position in ['QB', 'WR', 'TE', 'RB']:
        replacement_values[position] = df.loc[df['PLAYER'] == player].values[0, -1]
    
df['VOR'] = df.apply(
    lambda row: row['FPTS'] - replacement_values.get(row['POS']), axis=1
)

df = df.sort_values(by='VOR', ascending=False)
df['VALUERANK'] = df['VOR'].rank(ascending=False)

#with pd.option_context('display.max_rows', None):
    #display(df.head(100))
    
adp_df = make_adp_df()
#adp_df.head()

adp_df['ADPRANK']=adp_df['AVG'].rank(method='first')
#adp_df.head()

df = df.merge(adp_df, how='left', on=['PLAYER', 'POS'])
#df.head()

with pd.option_context('display.max_rows', None):
    display(df.head(100))

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK
0,Christian McCaffrey,RB,367.1,233.7,1.0,1.0,1.0
1,Saquon Barkley,RB,307.7,174.3,2.0,2.0,2.0
2,Ezekiel Elliott,RB,299.4,166.0,3.0,3.0,3.0
3,Alvin Kamara,RB,297.0,163.6,4.0,5.0,5.0
4,Michael Thomas,WR,327.1,161.5,5.0,5.0,4.0
5,Dalvin Cook,RB,290.0,156.6,6.0,6.0,7.0
6,Davante Adams,WR,292.8,127.2,7.0,8.5,8.0
7,Derrick Henry,RB,259.3,125.9,8.0,6.0,6.0
8,Clyde Edwards-Helaire,RB,255.6,122.2,9.0,16.5,15.0
9,Austin Ekeler,RB,250.2,116.8,10.0,21.5,23.0


In [16]:
df['SLEEPERSCORE'] = df['ADPRANK'] - df['VALUERANK']
df.loc[df['AVG']<160].sort_values(by='SLEEPERSCORE', ascending=False).head(15)

,PLAYER,POS,FPTS,VOR,VALUERANK,AVG,ADPRANK,SLEEPERSCORE
66,Antonio Gibson,RB,168.5,35.1,67.0,158.0,158.0,91.0
95,Preston Williams,WR,172.4,6.8,96.0,151.5,149.0,53.0
102,Duke Johnson,RB,135.0,1.6,103.0,156.0,156.0,53.0
110,DeSean Jackson,WR,162.3,-3.3,111.0,157.5,157.0,46.0
63,Tarik Cohen,RB,171.8,38.4,64.0,118.0,110.0,46.0
84,Christian Kirk,WR,181.9,16.3,85.0,125.0,123.0,38.0
78,Sterling Shepard,WR,188.4,22.8,79.0,119.5,116.0,37.0
72,Jamison Crowder,WR,194.1,28.5,73.0,118.0,109.0,36.0
58,James White,RB,173.0,39.6,59.0,98.5,94.0,35.0
113,Jared Goff,QB,256.2,-4.3,114.0,143.5,142.0,28.0
